Run FUTURES_preprocessing prior to running this script

In [ ]:
# Set up GRASS environment
import os
from IPython.display import Image
from helper import show_interactively, show
import grass.script as gs
import csv
# had to add shell = True to helper.py for this to work
from helper import initialize_GRASS_notebook
initialize_GRASS_notebook(binary='grass83',
                          grassdata="D:\\grassdata", location="FUTURES_zoning_prelim",
                          mapset="input")

In [ ]:
train_years = [2001, 2004, 2006, 2008, 2011]
validate_years = [2013, 2016, 2019, 2021]
# scenarios
# core district zones
core_district = 'dist_to_mixeduse_km,dist_to_nonresidential_km,dist_to_residential_km'
# sub-district zones
sub_district = 'dist_to_downtown_km,dist_to_planned_dev_km,dist_to_commercial_km,dist_to_industrial_km,dist_to_low_den_residential_km,dist_to_high_den_residential_km'

# without zoning
parameters = 'devpressure_30_05_01_2001,slope,road_dens_perc,forest_2001_smooth_perc,dist_to_water_km,dist_to_protected_km,dist_to_city_boundaries_km'
# with zoning - core
core_parameters = ",".join([parameters, core_district])
# with zoning - sub
sub_parameters = ",".join([parameters, sub_district])
# with core and sub zoning districts
all_parameters = ",".join([parameters, core_district, sub_district])
# development change raster
developed = 'urban_change_01_11'

# subregions
subregion = 'sa_counties'

In [13]:
# Function to get predictors from potential.csv file
# Note: not removing first three columns (ID, Intercept, Devpressure) in this version of the function but do in FUTURES_PGA.ipynb
def get_pred(file_name):
  file = open(file_name)
  csvreader = csv.reader(file)
  pred = []
  pred = next(csvreader)
  file.close()
  return pred

POTENTIAL without zoning

In [ ]:
gs.run_command('r.futures.potential', flags='d', input='sampling',
               output='potential.csv', columns=parameters,
               developed_column=developed, subregions_column=subregion, min_variables=6)

In [14]:
# get list of predictors from first row in potential.csv
pred = get_pred('potential.csv')
pred

['devpressure_30_05_01_2001',
 'slope',
 'forest_2001_smooth_perc',
 'dist_to_water_km',
 'dist_to_mixeduse_km',
 'dist_to_nonresidential_km',
 'dist_to_residential_km']

In [ ]:
gs.run_command('r.futures.potsurface', input='potential.csv', subregions=subregion, output='suitability')
gs.run_command('r.colors', map='suitability', color='byr')
show('suitability')

POTENTIAL with core zoning

In [ ]:
gs.run_command('r.futures.potential', flags='d', input='sampling',
               output='potential_core.csv', columns=core_parameters,
               developed_column=developed, subregions_column=subregion, min_variables=6)

In [ ]:
gs.run_command('r.futures.potsurface', input='potential_core.csv', subregions=subregion, output='suitability_core')
gs.run_command('r.colors', map='suitability_core', color='byr')
show('suitability_core')

POTENTIAL with sub zoning

In [ ]:
gs.run_command('r.futures.potential', flags='d', input='sampling',
               output='potential_sub.csv', columns=sub_parameters,
               developed_column=developed, subregions_column=subregion, min_variables=6)

In [ ]:
gs.run_command('r.futures.potsurface', input='potential_sub.csv', subregions=subregion, output='suitability_sub')
gs.run_command('r.colors', map='suitability_sub', color='byr')
show('suitability_sub')

POTENTIAL given core and sub zoning

In [ ]:
gs.run_command('r.futures.potential', flags='d', input='sampling',
               output='potential_all.csv', columns=all_parameters,
               developed_column=developed, subregions_column=subregion, min_variables=6)

In [ ]:
gs.run_command('r.futures.potsurface', input='potential_all.csv', subregions=subregion, output='suitability_all')
gs.run_command('r.colors', map='suitability_all', color='byr')
show('suitability_all')